<a href="https://colab.research.google.com/github/JNarimatsu/Bootcamp_Avanti_2025.3/blob/main/Entrega_02_An%C3%A1lise_comparativa_de_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise comparativa de modelos
 - Conjunto de dados: `laptop_price` (Preços laptops)
 - Cientistas de dados:
   - Juliana Narimatsu

---

In [1]:
# @title Preparação de ambiente
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# pipelines e transformadores
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.compose import ColumnTransformer

# codificação de variáveis
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.feature_extraction.text import CountVectorizer

# normalização
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

# dados faltantes
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer

# modelagem
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate, KFold, ShuffleSplit, RandomizedSearchCV
from sklearn.dummy import DummyRegressor # Import DummyRegressor
from sklearn.ensemble import RandomForestRegressor # Import RandomForestRegressor


#Organização e formatação
from IPython.display import display, Markdown, HTML


In [2]:
# @title Leitura do conjunto e criação do dicionário de dados

df_prices_latop = pd.read_csv(
    'https://raw.githubusercontent.com/JNarimatsu/Bootcamp_Avanti_2025.3/main/price_laptop_final.csv', encoding='latin-1', sep=','
)

# @title Dicionário de dados
df_dict_prices = pd.DataFrame([
    {
        "variavel": "laptop_ID",
        "descricao": "Identificação do laptop",
        "tipo": "quantitativa",
        "subtipo": "discreta",
    },
    {
        "variavel": "Company",
        "descricao": "Fabricante do laptop",
        "tipo": "qualitativa",
        "subtipo": "nominal",
    },
    {
        "variavel": "Product",
        "descricao": "Modelo do laptop",
        "tipo": "qualitativa",
        "subtipo": "nominal",
    },
    {
        "variavel": "TypeName",
        "descricao": "Tipo de Laptop",
        "tipo": "qualitativa",
        "subtipo": "nominal",
    },
    {
        "variavel": "Inches",
        "descricao": "Polegadas",
        "tipo": "quantitativa",
        "subtipo": "contínua",
    },
    { #Em nova análise foi decidido colocar a variável ScreenResolution como nominal
        "variavel": "ScreenResolution",
        "descricao": "Polegadas da tela",
        "tipo": "qualitativa",
        "subtipo": "nominal",
    },
    {
        "variavel": "CPU_Brand",
        "descricao": "Qual cpu utilizada no laptop",
        "tipo": "qualitativa",
        "subtipo": "ordinal",
    },
     {
        "variavel": "Ram",
        "descricao": "Quantidade de memória ram em gigas",
        "tipo": "qualitativa",
        "subtipo": "ordinal",
    },
    { #em nova análise foi decidido colocar a variável Memory como nominal
        "variavel": "Memory",
        "descricao": "Quantidade de memória em gigas",
        "tipo": "qualitativa",
        "subtipo": "nominal",
    },
    { #em nova análise foi decidido colocar a variável GPU como nominal
        "variavel": "Gpu",
        "descricao": "Qual gpu utilizada no laptop",
        "tipo": "qualitativa",
        "subtipo": "nominal",
    },
    {
        "variavel": "OpSys",
        "descricao": "Qual sistema operacional o laptop utiliza",
        "tipo": "qualitativa",
        "subtipo": "nominal",
    },
    {
      "variavel": "Weight",
        "descricao": "Peso do laptop",
        "tipo": "quantitativa",
        "subtipo": "contínua",
    },
    {
        "variavel": "Price_euros",
        "descricao": "Preço do laptop",
        "tipo": "quantitativa",
        "subtipo": "contínua",
    }
])
df_dict_prices

,variavel,descricao,tipo,subtipo
0,laptop_ID,Identificação do laptop,quantitativa,discreta
1,Company,Fabricante do laptop,qualitativa,nominal
2,Product,Modelo do laptop,qualitativa,nominal
3,TypeName,Tipo de Laptop,qualitativa,nominal
4,Inches,Polegadas,quantitativa,contínua
5,ScreenResolution,Polegadas da tela,qualitativa,nominal
6,CPU_Brand,Qual cpu utilizada no laptop,qualitativa,ordinal
7,Ram,Quantidade de memória ram em gigas,qualitativa,ordinal
8,Memory,Quantidade de memória em gigas,qualitativa,nominal
9,Gpu,Qual gpu utilizada no laptop,qualitativa,nominal


In [3]:
# @title Verificação de dados faltantes

df_prices_latop.isnull().sum()

,0
laptop_ID,0
Company,0
Product,0
TypeName,0
Inches,0
ScreenResolution,0
Ram,0
Memory,0
Gpu,0
OpSys,0


---

Não existem dados faltantes

---

In [4]:
# @title Seleção de variáveis e separação de entradas e saídas

#Variável-alvo
target_variable = ['Price_euros']

# Variáveis inúteis
useless_variables =  (
    df_dict_prices
    .query("tipo == 'inútil'")
    .variavel
    .to_list()
)
# Variáveis Nominais
unused_variables = useless_variables + target_variable
nominal_variables = (
    df_dict_prices
    .query("subtipo == 'nominal' and variavel not in @unused_variables")
    .variavel
    .to_list()
)
# Variáveis Ordinais
ordinal_variables = (
    df_dict_prices
    .query("subtipo == 'ordinal' and variavel not in @unused_variables")
    .variavel
    .to_list()
)
# Variáveis Continuas
continuous_variables = (
    df_dict_prices
    .query("subtipo == 'contínua' and variavel not in @unused_variables")
    .variavel
    .to_list()
)
# Variáveis Discretas
discrete_variables = (
    df_dict_prices
    .query("subtipo == 'discreta' and variavel not in @unused_variables")
    .variavel
    .to_list()
)

display(Markdown(
    f"- **Variável alvo:** {target_variable} \n\n"
    f"- **Variáveis qualitativas nominais:** {nominal_variables} \n"
    f"- **Variáveis qualitativas ordinais:** {ordinal_variables} \n"
    f"- **Variáveis quantitativas contínuas:** {continuous_variables} \n"
    f"- **Variáveis quantitativas discretas:** {discrete_variables} \n"

    "---"
))

X = df_prices_latop.drop(columns=unused_variables)
y = df_prices_latop[target_variable]

- **Variável alvo:** ['Price_euros'] 

- **Variáveis qualitativas nominais:** ['Company', 'Product', 'TypeName', 'ScreenResolution', 'Memory', 'Gpu', 'OpSys'] 
- **Variáveis qualitativas ordinais:** ['CPU_Brand', 'Ram'] 
- **Variáveis quantitativas contínuas:** ['Inches', 'Weight'] 
- **Variáveis quantitativas discretas:** ['laptop_ID'] 
---

## Preparação de dados

Cada um dos tipos de variáveis foi submetido a um fluxo de tratamento de dados específico, a saber:

### Variáveis quantitativas
 - **Contínuas**: imputação de valores faltantes através da média e normalização min-max.
 - **Discretas**: imputação de valores faltatnes através da mediana e normalização min-max.

### Variáveis qualitativas
 - **Ordinais**:  imputação de valores faltantes a partir do valor mais frequente e codificação ordinal de valores.
 - **Nominais**: imputação de valores faltantes através da moda e codificação via *one-hot encoding*.

In [5]:
# variables discretas
discrete_preprocessing = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='median')), # tratamento de dados faltantes
    ("normalization",  MinMaxScaler())# normalização
])
# variables continuas
continuous_preprocessing = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')), # tratamento de dados faltantes
    ("normalization",  MinMaxScaler())# normalização
])
# variables ordinal
ordinal_preprocessing = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ("encoding", OrdinalEncoder(
        categories=[
            ['2GB', '4GB', '6GB', '8GB', '12GB', '16GB', '24GB', '32GB', '64GB'],
            ['Intel Atom X5', 'Intel Celeron', 'Intel Pentium', 'Intel Core i3',
             'Intel Core i5', 'Intel Core i7', 'Intel Core', 'Intel Xeon', 'AMD']
        ],
        handle_unknown='use_encoded_value',  # IMPORTANTE: tratar categorias desconhecidas
        unknown_value=-1                     # Valor para categorias não vistas durante o treino
    ))
])
# variables nominais
nominal_preprocessing = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ("encoding", OneHotEncoder(sparse_output=False, handle_unknown='ignore')), # Codificação de variáveis
    ('normalization', StandardScaler()) # Normalização de dados (Remove a média (centraliza em 0), Escala para variância unitária (desvio padrão = 1))

])

## Validação cruzada

Iremos análisar quatro modelos, que serão testados utilizando a validação cruzada de monte-carlo com 30 repetições. Os modelos utilizados na análise são:

 - Regressão linear (Linear Regression)
 - K-vizinhos mais próximos (K-Nearest-Neighbors)
 - Máquinas de vetores-suporte (Support Vector Machine)
 - Árvores de decisão (Decision Tree)

Além disso, cada um desses algoritmos será testado com diferentes hiper-parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo. Tal otimização será realizada utilizando com um validação cruzada k-fold a partir dos dados de treinamento.

Utilizaremos as seguintes métricas para análise:
 - **Erro mério absoluto (*mean absolute error*, MAE)**: mede o erro médio absoluto entre valores reais e previstos. Tem interpretação direta em unidades da variável alvo.
 - **Erro médio quadrático (*mean squared error*, MSE)**: penaliza mais fortemente os grandes erros (pois eleva ao quadrado).
 - **Coeficiente de determinação (R² score)**: mede quanto o modelo consegue reduzir o erro comparado a média dos dados. Varia entre menos infinito até 1. Um bom modelo tende a ter valores próximos de 1.
 - **Erro percentual médio absoluto (*mean absolute percentage error*, MAPE)**: mede o erro percentual médio. É mais indicado quando temos valores de diferentes escalas.

In [6]:
# @title Validação de modelos de machine Learning

models = [LinearRegression(), KNeighborsRegressor(n_neighbors=5), SVR(), RandomForestRegressor(random_state=42)]
metrics = [
    'neg_mean_absolute_error',
    'neg_mean_squared_error',
    'neg_mean_absolute_percentage_error',
    'r2',
]
monte_carlo = ShuffleSplit(n_splits=10, test_size=.2, random_state=42)

ValidaçAo de modelos de machine learning utilizando o metodo de monte carlo. Nessa validação estamos criando 4 modelos diferentes para comparação, sendo:
- Linear regretion (modelo linear simples)
- K-Neighbors Regressor (k=5)
- SVR: Maquina de vetores de suporte para regressão
- Random Forest Regressor: Floresta aleatória (com random_state para reprodutibilidade)

In [7]:
preprocessing = ColumnTransformer([
    ("nominal", nominal_preprocessing, nominal_variables),
    ("ordinal", ordinal_preprocessing, ordinal_variables),
    ("continuous", continuous_preprocessing, continuous_variables),
    ("discrete", discrete_preprocessing, discrete_variables),
])

In [8]:
# @title Implementação dos modelos
preprocessing = ColumnTransformer(transformers=[
    ("ordinal", ordinal_preprocessing, ordinal_variables),
    ("nominal", nominal_preprocessing, nominal_variables),
    ("discrete", discrete_preprocessing, discrete_variables),
    ("continuous", continuous_preprocessing, continuous_variables),
], remainder='passthrough')

modelos = [
    {
        "nome": "LRG",
        "objeto": LinearRegression(),
        "hp": {}
    }, {
        "nome": "KNN",
        "objeto": KNeighborsRegressor(),
        "hp": {
            'n_neighbors': np.arange(1, 31, 5), # Número de vizinhos entre 1 e 30
            'weights': ['uniform', 'distance'], # Peso uniforme ou baseado na distância
            'p': [1, 2] # Distância de Manhattan (p=1) ou Euclidiana (p=2)
        }
    }, {
        "nome": "DTR",
        "objeto": DecisionTreeRegressor(random_state=42),
        "hp": {
            'max_depth': [None] + list(np.arange(2, 20, 4)), # Profundidade máxima
            'max_features': [None, 'sqrt', 'log2'] # Máximo de features
        }
    }, {
        "nome": "SVR",
        "objeto": SVR(),
        "hp": {
            'C': np.logspace(-3, 3, 10),     # Regularização
            'epsilon': np.logspace(-4, 0, 10),  # Insensibilidade à margem
            'kernel': ['linear', 'rbf', 'sigmoid'],  # Kernel a ser usado
        }
    }
]

##  Comparação de modelos com Hyperparametro

- Linear Regression (LRG) : Modelo linear simples sem hyperparâmetro para otimizar
- K-Nearest Neighbores (KNN)
- Decision Tree Regressor (DTR)
- Suport Vectr regressor (SVR)


In [9]:
# @title Aplicação da validação cruzada
cv = ShuffleSplit(n_splits=30, test_size=.2, random_state=42)
metrics = {
    'neg_mean_absolute_error': 'MAE',
    'neg_mean_squared_error': 'MSE',
    'r2': 'R2',
    'neg_mean_absolute_percentage_error': 'MAPE'
}

results = []
for modelo in modelos:
    random_search = RandomizedSearchCV(
        estimator=modelo["objeto"],
        param_distributions=modelo["hp"],
        n_iter=1 if modelo["nome"] == "LRG" else 10,
        scoring='neg_mean_squared_error',
        cv=5,
        random_state=42,
        n_jobs=-1
    )
    approach = Pipeline(steps=[
        ("preprocessing", preprocessing),
        ("model", random_search)
    ])
    metric_results = cross_validate(approach, X=X, y=y.values.ravel(), cv=cv, scoring=list(metrics.keys()))
    metric_results['modelo'] = [modelo["nome"]] * len(metric_results['fit_time'])
    results.append(pd.DataFrame(metric_results))
final_results = pd.concat(results, axis=0)

In [10]:
# @title Apresentação de resultados

# função para hilight de melhores resultados
def highlight_best(s, props=''):
    if s.name[1] != 'std':
        if s.name[0].endswith('time'):
            return np.where(s == np.min(s.values), props, '')
        if s.name[0].endswith('R2'):
            return np.where(s == np.max(s.values), props, '')
        return np.where(s == np.min(s.values), props, '')

# atualização de valores das métricas
for metric in metrics.keys():
    if 'neg' in metric:
        final_results[f"test_{metric}"] *= -1


# apresentação de resultados
(
    final_results
    .rename(columns={f"test_{name}": value for name, value in metrics.items()})
    .groupby("modelo").agg(["mean", "std"]).T
    .style
    .apply(highlight_best, props='color:white;background-color:gray;font-weight: bold;', axis=1)
    .set_table_styles([{'selector': 'td', 'props': 'text-align: center;'}])
)

## Conclusão

- **Linear Regression (LRG):** Este modelo apresentou um bom desempenho geral, com um R² médio de aproximadamente 0.73. As métricas de erro (MAE, MSE, MAPE) indicam uma precisão razoável nas previsões. O desvio padrão das métricas sugere uma consistência aceitável entre as diferentes divisões da validação cruzada.

- **K-Nearest Neighbors (KNN):** O modelo KNN teve um desempenho inferior em comparação com LRG e SVR, com um R² médio significativamente menor (aproximadamente 0.53) e valores de erro maiores. Isso pode indicar que a abordagem baseada na proximidade dos vizinhos não capturou bem a relação nos dados para este problema.

- **Decision Tree Regressor (DTR):** O DTR apresentou um desempenho intermediário, com um R² médio em torno de 0.68. Embora melhor que o KNN, ainda ficou aquém do LRG e SVR. As métricas de erro também refletem essa performance intermediária.

- **Support Vector Regressor (SVR):** O SVR demonstrou o melhor desempenho médio entre todos os modelos avaliados, alcançando o maior R² médio (aproximadamente 0.74) e os menores valores médios para todas as métricas de erro (MAE, MSE, MAPE). Isso sugere que o SVR foi o mais eficaz em capturar os padrões nos dados para prever o preço dos laptops. O desvio padrão para o SVR é comparável ou ligeiramente maior que o do LRG em algumas métricas, mas a sua performance média superior o destaca como o modelo com melhor poder preditivo neste conjunto de dados.

Em resumo, o modelo **SVR** foi o que apresentou o melhor desempenho preditivo médio para este problema, seguido de perto pelo **Linear Regression**. Os modelos KNN e DTR não foram tão eficazes. A escolha final entre SVR e LRG pode depender de outros fatores, como tempo de treinamento e interpretabilidade, mas em termos de acurácia preditiva média nas métricas avaliadas, o SVR se destacou.